In [2]:
import numpy as np
import tensorflow as tf

from hyperspherical_vae.distributions import VonMisesFisher
from hyperspherical_vae.distributions import HypersphericalUniform
from __future__ import print_function
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math,random,re
import time
from scipy.stats import norm
from scipy.stats import spearmanr,pearsonr
from sklearn.preprocessing import normalize
import tensorflow as tf
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers
from keras.layers import LSTM, RepeatVector
from keras.layers import Input, Dense, Lambda, Dropout,Activation, TimeDistributed
from keras import backend as K
from keras import objectives
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras import regularizers

Using TensorFlow backend.


In [3]:
from keras import backend as K
from os import environ
from importlib import reload


# user defined function to change keras backend
def set_keras_backend(backend):
    if K.backend() != backend:
       environ['KERAS_BACKEND'] = backend
       reload(K)
       assert K.backend() == backend

# call the function with "theano"
set_keras_backend("tensorflow")

In [4]:
#Invariants
ORDER_KEY="XILVAGMFYWEDQNHCRKSTPBZ-"[::-1]
ORDER_LIST=list(ORDER_KEY)

In [5]:
%reload_ext autoreload
%autoreload 1
from helper_tools import *
from helper_tools_for_plotting import *

In [6]:
data=pdataframe_from_alignment_file("PABP_YEAST_hmmerbit_plmc_n5_m30_f50_t0.2_r115-210_id100_b48.a2m",50000)
print ("number of data points: ",len(data))
data_set_size=len(data)
data.head()

number of data points:  50000


,name,sequence
0,PABP_YEAST/115-210,qrdpslrkKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...
1,ur|UPI0004E53ABB|UniRef100_UPI0004E53ABB/203-294,..epangsPKFFNVYVKNLPEKYTDDDLKSEFEAFGEITSAVVVKD...
2,ur|UPI0004E53ABB|UniRef100_UPI0004E53ABB/534-625,..epangsPKFFNVYVKNLPEKYTDDDLKSEFEASGEITSAVVVKD...
3,ur|UPI0004E53ABB|UniRef100_UPI0004E53ABB/312-398,........IRGLNLYLKNLDDTIDDERLKELFRPFGTIISCKVMVD...
4,ur|UPI0004E53ABB|UniRef100_UPI0004E53ABB/33-112,........---ASLYVGDLDLSVTEGQLFDLFSQIGPVASVRVCRD...


In [7]:
print ("length of sequence:", len(data.iloc[0]["sequence"]))#, len(data.iloc[0]["seq"]))
print ("sample sequence: ", data.iloc[0]["sequence"])

length of sequence: 96
sample sequence:  qrdpslrkKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKEAIDALNGMLLNGQEIYVAPHLSRkerdsq


In [8]:
indices=index_of_non_lower_case_dot(data.iloc[0]["sequence"])
data["seq"]=list(map(prune_seq,data["sequence"]))
data.head()

,name,sequence,seq
0,PABP_YEAST/115-210,qrdpslrkKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,KGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATDENGKSKGF...
1,ur|UPI0004E53ABB|UniRef100_UPI0004E53ABB/203-294,..epangsPKFFNVYVKNLPEKYTDDDLKSEFEAFGEITSAVVVKD...,PKFFNVYVKNLPEKYTDDDLKSEFEAFGEITSAVVVKDENGNSRGF...
2,ur|UPI0004E53ABB|UniRef100_UPI0004E53ABB/534-625,..epangsPKFFNVYVKNLPEKYTDDDLKSEFEASGEITSAVVVKD...,PKFFNVYVKNLPEKYTDDDLKSEFEASGEITSAVVVKDENGNSRGF...
3,ur|UPI0004E53ABB|UniRef100_UPI0004E53ABB/312-398,........IRGLNLYLKNLDDTIDDERLKELFRPFGTIISCKVMVD...,IRGLNLYLKNLDDTIDDERLKELFRPFGTIISCKVMVDSQGQSKGS...
4,ur|UPI0004E53ABB|UniRef100_UPI0004E53ABB/33-112,........---ASLYVGDLDLSVTEGQLFDLFSQIGPVASVRVCRD...,---ASLYVGDLDLSVTEGQLFDLFSQIGPVASVRVCRDIRRVSLGY...


In [9]:
print ("pruned sequence length:", len(data.iloc[0]["seq"]))
PRUNED_SEQ_LENGTH=len(data.iloc[0]["seq"])

pruned sequence length: 82


In [10]:
with open ("PABP_YEAST_hmmerbit_t0.2_r50000.reweight","rb") as to_read:
    new_weights=np.load(to_read)

#new_weights=reweight_sequences(data["seq"],0.1)
len(new_weights),new_weights[:10]

(50000, array([0.03125   , 0.2       , 0.2       , 0.2       , 0.16666667,
        0.2       , 0.16666667, 0.16666667, 0.0625    , 1.        ]))

In [11]:
#Encode training data in one_hot vectors
training_data_one_hot=[]
labels=[]
for i, row in data.iterrows():
    training_data_one_hot.append(translate_string_to_one_hot(row["seq"],ORDER_LIST))
print (len(training_data_one_hot))
#plt.imshow(training_data_one_hot[0],cmap="Greys")
training_data=np.array([np.array(list(sample.flatten())).T for sample in training_data_one_hot])
print(training_data.shape)


50000
(50000, 1968)


In [12]:
exp_data_full=pd.read_csv(
    "PABP_YEAST_Fields2013-singles.csv", sep=";", comment="#"
)
print ("number of mutants: ",len(exp_data_full))
exp_data_full.head()
exp_data_full.iloc[87]

number of mutants:  1188


mutant                               K131V
effect_prediction_epistatic         -10.06
effect_prediction_independent     -3.36052
linear                           0.0425856
Name: 87, dtype: object

In [13]:
exp_data_full.corr(method="spearman")

,effect_prediction_epistatic,effect_prediction_independent,linear
effect_prediction_epistatic,1.000000,0.745161,0.592781
effect_prediction_independent,0.745161,1.000000,0.423680
linear,0.592781,0.423680,1.000000


In [14]:
OFFSET=117
#Deciding offset requires investigating the dataset and alignment.
exp_data_singles=pd.DataFrame(columns=exp_data_full.columns)
#decide starting index depending on how the file is "headered"
for i,row in exp_data_full[1:].iterrows():
        pos=re.split(r'(\d+)', row.mutant) 
        if int(pos[1])-OFFSET in indices:
            exp_data_singles=exp_data_singles.append(row)
exp_data_singles=exp_data_singles.reset_index()
target_values_singles=list(exp_data_singles["linear"])
exp_data_singles.head(10) 

,index,mutant,effect_prediction_epistatic,effect_prediction_independent,linear
0,1,G126C,-5.663638,-0.027602,0.449027
1,2,G126E,-6.611062,-1.827612,0.588928
2,3,G126D,-7.270577,-1.180094,0.229853
3,4,G126N,-5.809167,0.387443,0.679435
4,5,G126S,-4.617248,0.661686,0.721788
5,6,G126R,-5.582381,1.144148,0.313690
6,7,G126W,-8.079901,-2.052391,0.226032
7,8,G126V,-5.435631,-0.800307,0.230315
8,9,N127A,-4.987206,-2.251823,0.062650
9,10,N127C,-8.044181,-2.215639,0.024996


In [15]:
mutation_data=[re.split(r'(\d+)', s) for s in exp_data_singles.mutant]
wt_sequence=data.iloc[0].seq
mutants=mutate_single(wt_sequence,mutation_data,offset=0,index=3) #note that you change index to 1

#sanity checks
print (len(mutants),len(exp_data_singles))
#the mutant should be in the correct place
print (list(zip(wt_sequence,mutants[3]))[:10])

1187 1187
[('K', 'K'), ('G', 'G'), ('S', 'S'), ('G', 'N'), ('N', 'N'), ('I', 'I'), ('F', 'F'), ('I', 'I'), ('K', 'K'), ('N', 'N')]


In [16]:
#Test data with wt at 0 index
one_hot_mutants=[]
mutants_plus=[data.iloc[0]["seq"]]+mutants
for mutant in mutants_plus:
    one_hot_mutants.append(translate_string_to_one_hot("".join(mutant),ORDER_LIST))

test_data_plus=np.array([np.array(list(sample.flatten())).T for sample in one_hot_mutants])

In [17]:
exp_data_full=pd.read_csv(
    "PABP_YEAST_Fields2013-doubles.csv", sep=";", comment="#"
)
print ("number of mutants: ",len(exp_data_full))
exp_data_full.head()
exp_data_full.iloc[0]

number of mutants:  36522


mutant                           G169W,F170V
effect_prediction_epistatic          -18.161
effect_prediction_independent       -15.1525
XY_Enrichment_score                  0.05916
Name: 0, dtype: object

In [18]:
exp_data_full.corr(method="spearman")

,effect_prediction_epistatic,effect_prediction_independent,XY_Enrichment_score
effect_prediction_epistatic,1.000000,0.715371,0.620022
effect_prediction_independent,0.715371,1.000000,0.497743
XY_Enrichment_score,0.620022,0.497743,1.000000


In [19]:
OFFSET=160
#Deciding offset requires investigating the dataset and alignment.
exp_data_doubles=pd.DataFrame(columns=exp_data_full.columns)
#decide starting index depending on how the file is "headered"
for i,row in exp_data_full[0:].iterrows():
        pos=re.split(r'(\d+)', row.mutant) 
        if int(pos[1])-OFFSET in indices and int(pos[3])-OFFSET in indices:
            exp_data_doubles=exp_data_doubles.append(row)
exp_data_doubles=exp_data_doubles.reset_index()
exp_data_doubles.head(5)

,index,mutant,effect_prediction_epistatic,effect_prediction_independent,XY_Enrichment_score
0,0,"G169W,F170V",-18.161003,-15.152495,0.059160
1,1,"G169V,F170V",-13.753099,-8.845704,0.045765
2,2,"G169A,F170I",-9.115749,-3.954389,0.075799
3,3,"G169C,F170Y",-9.761077,-4.004533,0.700485
4,4,"G169A,F170S",-10.212144,-5.743002,0.061518


In [20]:
target_values_doubles=list(exp_data_doubles["XY_Enrichment_score"])
exp_data_doubles.corr(method="spearman")

,index,effect_prediction_epistatic,effect_prediction_independent,XY_Enrichment_score
index,1.000000,0.143941,0.140272,0.117997
effect_prediction_epistatic,0.143941,1.000000,0.773236,0.520277
effect_prediction_independent,0.140272,0.773236,1.000000,0.488397
XY_Enrichment_score,0.117997,0.520277,0.488397,1.000000


In [21]:
mutation_data1=[re.split(r'(\d+)', s.split(",")[0]) for s in exp_data_doubles.mutant]
mutation_data2=[re.split(r'(\d+)', s.split(",")[1]) for s in exp_data_doubles.mutant]
wt_sequence=data.iloc[0].seq

mutants_double=mutate_double(wt_sequence,mutation_data1,mutation_data2,offset=0,index=46)

#sanity checks
print (len(mutants_double),len(exp_data_doubles))
#the mutant should be in the correct place
print (list(zip(wt_sequence,mutants_double[2]))[40:50])

13876 13876
[('G', 'G'), ('K', 'K'), ('S', 'S'), ('K', 'K'), ('G', 'G'), ('F', 'F'), ('G', 'A'), ('F', 'I'), ('V', 'V'), ('H', 'H')]


In [22]:
#Test data with wt at 0 index
one_hot_mutants=[]
mutants_plus=[data.iloc[0]["seq"]]+mutants_double
for mutant in mutants_plus:
    one_hot_mutants.append(translate_string_to_one_hot("".join(mutant),ORDER_LIST))

test_data_doubles_plus=np.array([np.array(list(sample.flatten())).T for sample in one_hot_mutants])

In [23]:
all_test_data=np.vstack([test_data_plus,test_data_doubles_plus[1:]])
all_test_data_flattened=np.array([np.array(list(sample.flatten())).T for sample in all_test_data])

In [24]:
class rho_vs_mutants(keras.callbacks.Callback):
    def __init__(self,mutants,test_set_size,aa_size,sequence_size):
        self.mutants=mutants
        self.sample_size=test_set_size
        self.aa_size=aa_size
        self.sequence_size=sequence_size
        self.scores=[]
        self.count_batch=0
    def on_train_begin(self, logs={}):
        self.losses = []
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
    #This allows us to track the "progress" of the model on different epochs
    def on_epoch_end(self,batch,logs):
        x_decoded=vae.predict(test_data_plus[0:self.sample_size],batch_size=batch_size)
        digit = x_decoded[0].reshape(self.aa_size,self.sequence_size)
        digit_wt = normalize(digit,axis=0, norm='l1')
        wt_prob=compute_log_probability(digit,digit_wt)
        fitnesses=[]
        for sample in range(1,self.sample_size):
            digit = x_decoded[sample].reshape(self.aa_size,self.sequence_size)
            digit = normalize(digit,axis=0, norm='l1')
            fitness=compute_log_probability(test_data_plus[sample].reshape(self.aa_size,self.sequence_size),digit)-wt_prob
            fitnesses.append(fitness)
        print (","+str(spearmanr(fitnesses,target_values_singles[:self.sample_size-1])))
        self.scores.append(spearmanr(fitnesses,target_values_singles[:self.sample_size-1])[0])
            

In [25]:
class ModelVAE(object):

    def __init__(self, x, h_dim, z_dim, activation=tf.nn.relu, distribution='normal'):
        """
        ModelVAE initializer

        :param x: placeholder for input
        :param h_dim: dimension of the hidden layers
        :param z_dim: dimension of the latent representation
        :param activation: callable activation function
        :param distribution: string either `normal` or `vmf`, indicates which distribution to use
        """
        self.x, self.h_dim, self.z_dim, self.activation, self.distribution = x, h_dim, z_dim, activation, distribution

        self.z_mean, self.z_var = self._encoder(self.x)

        if distribution == 'normal':
            self.q_z = tf.distributions.Normal(self.z_mean, self.z_var)
        elif distribution == 'vmf':
            self.q_z = VonMisesFisher(self.z_mean, self.z_var)
        else:
            raise NotImplemented

        self.z = self.q_z.sample()

        self.logits = self._decoder(self.z)

    def _encoder(self, x):
        """
        Encoder network

        :param x: placeholder for input
        :return: tuple `(z_mean, z_var)` with mean and concentration around the mean
        """
        # dynamic binarization
        x = tf.cast(tf.greater(x, tf.random_uniform(shape=tf.shape(x), dtype=x.dtype)), dtype=x.dtype)
        
        # 2 hidden layers encoder
        h0 = tf.layers.dense(x, units=self.h_dim * 2, activation=self.activation)
        h1 = tf.layers.dense(h0, units=self.h_dim, activation=self.activation)

        if self.distribution == 'normal':
            # compute mean and std of the normal distribution
            z_mean = tf.layers.dense(h1, units=self.z_dim, activation=None)
            z_var = tf.layers.dense(h1, units=self.z_dim, activation=tf.nn.softplus)
        elif self.distribution == 'vmf':
            # compute mean and concentration of the von Mises-Fisher
            z_mean = tf.layers.dense(h1, units=self.z_dim, activation=lambda x: tf.nn.l2_normalize(x, axis=-1))
            # the `+ 1` prevent collapsing behaviors
            z_var = tf.layers.dense(h1, units=1, activation=tf.nn.softplus) + 1
        else:
            raise NotImplemented

        return z_mean, z_var

    def _decoder(self, z):
        """
        Decoder network

        :param z: tensor, latent representation of input (x)
        :return: logits, `reconstruction = sigmoid(logits)`
        """
        # 2 hidden layers decoder
        h2 = tf.layers.dense(z, units=self.h_dim, activation=self.activation)
        h2 = tf.layers.dense(h2, units=self.h_dim * 2, activation=self.activation)
        logits = tf.layers.dense(h2, units=self.x.shape[-1], activation=None)

        return logits


class OptimizerVAE(object):

    def __init__(self, model, learning_rate=1e-3):
        """
        OptimizerVAE initializer

        :param model: a model object
        :param learning_rate: float, learning rate of the optimizer
        """

        # binary cross entropy error
        self.bce = tf.nn.sigmoid_cross_entropy_with_logits(labels=model.x, logits=model.logits)
        self.reconstruction_loss = tf.reduce_mean(tf.reduce_sum(self.bce, axis=-1))

        if model.distribution == 'normal':
            # KL divergence between normal approximate posterior and standard normal prior
            self.p_z = tf.distributions.Normal(tf.zeros_like(model.z), tf.ones_like(model.z))
            kl = model.q_z.kl_divergence(self.p_z)
            self.kl = tf.reduce_mean(tf.reduce_sum(kl, axis=-1))
        elif model.distribution == 'vmf':
            # KL divergence between vMF approximate posterior and uniform hyper-spherical prior
            self.p_z = HypersphericalUniform(model.z_dim - 1, dtype=model.x.dtype)
            kl = model.q_z.kl_divergence(self.p_z)
            self.kl = tf.reduce_mean(kl)
        else:
            raise NotImplemented

        self.ELBO = - self.reconstruction_loss - self.kl

        self.train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(-self.ELBO)

        self.print = {'recon loss': self.reconstruction_loss, 'ELBO': self.ELBO, 'KL': self.kl}


def log_likelihood(model, optimizer, n=10):
    """

    :param model: model object
    :param optimizer: optimizer object
    :param n: number of MC samples
    :return: MC estimate of log-likelihood
    """

    z = model.q_z.sample(n)

    log_p_z = optimizer.p_z.log_prob(z)

    if model.distribution == 'normal':
        log_p_z = tf.reduce_sum(log_p_z, axis=-1)

    log_p_x_z = -tf.reduce_sum(optimizer.bce, axis=-1)

    log_q_z_x = model.q_z.log_prob(z)

    if model.distribution == 'normal':
        log_q_z_x = tf.reduce_sum(log_q_z_x, axis=-1)

    return tf.reduce_mean(tf.reduce_logsumexp(
        tf.transpose(log_p_x_z + log_p_z - log_q_z_x) - np.log(n), axis=-1))


In [32]:
# hidden dimension and dimension of latent space
H_DIM = 128
Z_DIM = 2
batch_size = 20
original_dim=len(ORDER_LIST)*PRUNED_SEQ_LENGTH
output_dim=len(ORDER_LIST)*PRUNED_SEQ_LENGTH
latent_dim = 2
intermediate_dim = 250
nb_epoch = 2
epsilon_std = 1.0
np.random.seed(42) 
dataset_size = 50000
# digit placeholder
x = tf.placeholder(tf.float32, shape=(None, 1968))

# normal VAE
modelN = ModelVAE(x=x, h_dim=H_DIM, z_dim=Z_DIM, distribution='normal')
optimizerN = OptimizerVAE(modelN)

# hyper-spherical VAE
modelS = ModelVAE(x=x, h_dim=H_DIM, z_dim=Z_DIM + 1, distribution='vmf')
optimizerS = OptimizerVAE(modelS)
x_train=training_data[:data_set_size] #this needs to be divisible by batch size and less than or equal to dataset size
x_train = x_train.astype('float32')
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))

session = tf.Session()
session.run(tf.global_variables_initializer())

print('##### Normal VAE #####')
for epich in range(nb_epoch):
    for i in range(1000):
    # training
        start = (i*batch_size) % dataset_size
        end = min(start+batch_size,dataset_size)
        session.run(optimizerS.train_step, {modelN.x:x_train[start:end]})

    # every 100 iteration plot validation
        if i % 100 == 0:
            print(i, session.run({**optimizerN.print}, {modelN.x: x_train[start:end]}))


print('##### Hyper-spherical VAE #####')
for epich in range(nb_epoch):

    for i in range(1000):
    # training
        start = (i*batch_size) % dataset_size
        end = min(start+batch_size,dataset_size)
        session.run(optimizerS.train_step, {modelS.x:x_train[start:end]})


    # every 100 iteration plot validation
        if i % 100 == 0:
            print(i, session.run({**optimizerS.print}, {modelS.x: x_train[start:end]}))


##### Normal VAE #####
0 {'KL': 0.12592505, 'ELBO': -1365.071, 'recon loss': 1364.9451}
100 {'KL': 0.11927996, 'ELBO': -1364.9526, 'recon loss': 1364.8334}
200 {'KL': 0.1630525, 'ELBO': -1365.2765, 'recon loss': 1365.1134}
300 {'KL': 0.14344679, 'ELBO': -1364.9447, 'recon loss': 1364.8013}
400 {'KL': 0.15069847, 'ELBO': -1364.9459, 'recon loss': 1364.7952}
500 {'KL': 0.1354092, 'ELBO': -1364.8993, 'recon loss': 1364.7639}
600 {'KL': 0.12574944, 'ELBO': -1364.894, 'recon loss': 1364.7683}
700 {'KL': 0.16001925, 'ELBO': -1364.9957, 'recon loss': 1364.8357}
800 {'KL': 0.12122164, 'ELBO': -1364.8813, 'recon loss': 1364.7601}
900 {'KL': 0.12666322, 'ELBO': -1365.1824, 'recon loss': 1365.0557}
0 {'KL': 0.12592505, 'ELBO': -1364.9617, 'recon loss': 1364.8357}
100 {'KL': 0.11927996, 'ELBO': -1364.794, 'recon loss': 1364.6747}
200 {'KL': 0.1630525, 'ELBO': -1365.0576, 'recon loss': 1364.8945}
300 {'KL': 0.14344679, 'ELBO': -1365.0338, 'recon loss': 1364.8904}
400 {'KL': 0.15069847, 'ELBO': -136